# Import Python Libraries

In [1]:
# import needed libraries
import pickle
import numpy as np
import matplotlib.pyplot as plt
import random
import cv2
#import skimage.morphology as morp
#from skimage.filters import rank
from sklearn.utils import shuffle
import csv
import os
import tensorflow as tf
from tensorflow.contrib.layers import flatten
from sklearn.metrics import confusion_matrix


import numpy as np
import tensorflow as tf
from tensorflow import keras
from matplotlib import pyplot
import matplotlib.image as mpimg


import math #test on server

In [2]:
from utility import *
import keras
from keras.models import Sequential 
from keras.layers import Input, Dense, Activation, Conv2D, MaxPooling2D, Flatten, Dropout

Using TensorFlow backend.


# Load Data Set

In [3]:
#load data in form of np array
data_b = np.load('images_benign_50.npy')
print("# benign:", data_b.shape)

data_m = np.load('images_mali_50.npy')
print("# malignant:", data_m.shape) 

dim = int(math.sqrt(data_b.shape[1] / 3 )) # dimension of the image
print("dimension of images:", dim)

FileNotFoundError: [Errno 2] No such file or directory: 'images_benign_50.npy'

In [ ]:
#vertically stack files
data = np.vstack((data_b,data_m))
print("original data shape:", data.shape)

# create labelset (benign = 1, malignant = 2)
label_b = np.ones((data_b.shape[0],1),dtype=np.uint8) # an array of all zeros
label_m = np.ones((data_m.shape[0],1),dtype=np.uint8) * 2 # an array of all ones
label = np.vstack((label_b,label_m)) # all label

## train test separation

In [ ]:
data_train, label_train, data_valid, label_valid, data_test, label_test = train_test_shuffled_separation(data, label, train_percent= 0.8)

## convert label (1/2) to one hot code

In [ ]:
label_train = label_to_one_hot(label_train)
label_valid = label_to_one_hot(label_valid)
label_test = label_to_one_hot(label_test)

## reshape image arrays to N x size x size x 3

In [ ]:
data_train = data_train.reshape((data_train.shape[0], dim, dim, 3))
data_valid = data_valid.reshape((data_valid.shape[0], dim, dim, 3))
data_test = data_test.reshape((data_test.shape[0], dim, dim, 3))

## normalize image arrays to 0 - 1 

In [ ]:
data_train = normalize_data(data_train)
data_valid = normalize_data(data_valid)
data_test = normalize_data(data_test)
print("image size:", data_train.shape[1:])

# Display images from dataset

In [ ]:
# function to display the images
def list_images(data, label, ylabel="", cmap=None):
    plt.figure(figsize=(15, 16))
    
    for i in range(6):
        plt.subplot(1, 6, i+1)
        indx = random.randint(0, len(data))
        
        #Use gray scale color map if there is only one channel
        cmap = 'gray' if len(data[indx].shape) == 2 else cmap
        plt.imshow(data[indx], cmap = cmap)
        
        #plt.xlabel(signs[dataset_y[indx]])
        #plt.ylabel(ylabel)
        
        plt.xticks([])
        plt.yticks([])
    #plt.tight_layout(pad=0, h_pad=0, w_pad=0)
    plt.show()

In [ ]:
# plot the sample images
list_images(data_train, label_train)
list_images(data_test, label_test)

# Data Preprocessing

In [ ]:
# convert images to gray scale
def gray_scale(image):
    return cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

# VGG-13 implemented in keras

In [ ]:
VGG13_model = Sequential()

#  input dimension is (dim, dim, 3)
#  block 1: 2x Conv + Maxpool
# 3 -> 32 feature maps
VGG13_model.add(Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(dim, dim, 3), name='block1_conv1'))
VGG13_model.add(Conv2D(32, (3,3), activation='relu', padding='same', name='block1_conv2' ))
VGG13_model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2), name='block1_pool'))
VGG13_model.add(Dropout(0.25)) #dropout = 0.25

# block 2: 2x Conv + Maxpool
# 32 -> 64 feature maps
VGG13_model.add(Conv2D(64, (3,3), activation='relu', padding='same', name='block2_conv1'))
VGG13_model.add(Conv2D(64, (3,3), activation='relu', padding='same', name='block2_conv2' ))
VGG13_model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2), name='block2_pool'))
VGG13_model.add(Dropout(0.25)) #dropout = 0.25

# block 3: 2x Conv + Maxpool
# 64 -> 128 feature maps
VGG13_model.add(Conv2D(128, (3,3), activation='relu', padding='same', name='block3_conv1'))
VGG13_model.add(Conv2D(128, (3,3), activation='relu', padding='same', name='block3_conv2' ))
VGG13_model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2), name='block3_pool'))
VGG13_model.add(Dropout(0.25)) #dropout = 0.25

# block 4: 2x Conv + Maxpool
# 128 -> 256 feature maps
VGG13_model.add(Conv2D(256, (3,3), activation='relu', padding='same', name='block4_conv1'))
VGG13_model.add(Conv2D(256, (3,3), activation='relu', padding='same', name='block4_conv2' ))
VGG13_model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2), name='block4_pool'))
VGG13_model.add(Dropout(0.25)) #dropout = 0.25

# block 5: 2x Conv + Maxpool
# 256 -> 256 feature maps
VGG13_model.add(Conv2D(256, (3,3), activation='relu', padding='same', name='block5_conv1'))
VGG13_model.add(Conv2D(256, (3,3), activation='relu', padding='same', name='block5_conv2' ))
VGG13_model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2), name='block5_pool'))
VGG13_model.add(Dropout(0.25)) #dropout = 0.25

# block 6: 3x FC
VGG13_model.add(Flatten(name='block6_flatten'))
VGG13_model.add(Dense(2048, activation='relu', name='block6_fc1'))
VGG13_model.add(Dropout(0.5)) #dropout = 0.5
VGG13_model.add(Dense(2048, activation='relu', name='block6_fc2'))
VGG13_model.add(Dropout(0.5)) #dropout = 0.5
VGG13_model.add(Dense(2, activation='sigmoid', name='predictions')) # output two classes

### compile the VGG-13 architecture

In [ ]:
# optimizer
# Adam
# AdaDelta
# SGD
SGD = keras.optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
VGG13_model.compile(optimizer='SGD', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
data_train.shape

for i in range(len(VGG13_model.layers)):
               print(VGG13_model.layers[i].output)

### start training VGG-13

In [ ]:
history = VGG13_model.fit(data_train, label_train, batch_size=32, epochs=40, validation_data=(data_valid, label_valid))

### evaluate trained VGG-13 model

In [ ]:
score, acc = VGG13_model.evaluate(data_test, label_test, batch_size=32)
print('Test score:', score)
print('Test accuracy:', acc)

In [ ]:
plot_history(history)

# Lenet-5 implemented in keras

![LeNet-5](https://cdn-images-1.medium.com/max/1500/1*1TI1aGBZ4dybR6__DI9dzA.png)

In [ ]:
LeNet5_model = Sequential()

#  input dimension is (dim, dim, 3)
#  block 1: Conv + Maxpool
# 3 -> 6 feature maps
LeNet5_model.add(Conv2D(6, (3,3), activation='relu', padding='valid', input_shape=(dim, dim, 3), name='block1_conv1'))
LeNet5_model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2), name='block1_pool'))

# block 2: Conv + Maxpool
# 6 -> 16 feature maps
LeNet5_model.add(Conv2D(16, (3,3), activation='relu', padding='same', name='block2_conv1'))
LeNet5_model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2), name='block2_pool'))

# block 3: 3x FC
LeNet5_model.add(Flatten(name='block6_flatten'))
LeNet5_model.add(Dense(120, activation='relu', name='block3_fc1'))
LeNet5_model.add(Dense(84, activation='relu', name='block3_fc2'))
LeNet5_model.add(Dense(2, activation='sigmoid', name='predictions')) # output two classes

### compile LeNet-5 architecture

In [ ]:
# optimizer
# Adam
# AdaDelta
# SGD
LeNet5_model.compile(optimizer='AdaDelta', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
data_train.shape

In [ ]:
for i in range(len(LeNet5_model.layers)):
               print(LeNet5_model.layers[i].output)

### start training LeNet-5

In [ ]:
history = LeNet5_model.fit(data_train, label_train, batch_size=32, epochs=40, validation_data=(data_valid, label_valid))

### evaluate trained LeNet-5 model

In [ ]:
score, acc = LeNet5_model.evaluate(data_test, label_test, batch_size=32)
print('Test score:', score)
print('Test accuracy:', acc)

In [ ]:
plot_history(history)